# Import Library

In [48]:
import os
import tfx
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from tfx.components import CsvExampleGen
from tfx.components import StatisticsGen
from tfx.components import SchemaGen
from tfx.components import ExampleValidator
from tfx.components import Transform
from tfx.components import Trainer
from tfx.components import Evaluator
from tensorflow_model_analysis.proto import config_pb2
from tfx.components import Pusher
from tfx.proto import trainer_pb2
from tfx.proto import example_gen_pb2
from tfx.proto import pusher_pb2
from tfx.orchestration import pipeline
from tfx.dsl.experimental import latest_blessed_model_resolver
from tfx.types import Channel
from tfx.types.standard_artifacts import Model, ModelBlessing
from tfx.types import channel_utils
from tfx.types.standard_artifacts import ExternalArtifact
import tensorflow_transform as tft
from tfx.components import Transform
from tensorflow_transform import TFTransformOutput
import tensorflow as tf
from tfx.components.trainer.executor import GenericExecutor
from tfx.components.base import executor_spec
from tfx.dsl.experimental.latest_blessed_model_resolver import LatestBlessedModelResolver

Insight: 
* Import library yang dibutuhkan untuk project ini 

# Inisiasi Pipeline (Setup)

In [2]:
PIPELINE_NAME = "insurance_pipeline"
PIPELINE_ROOT = os.path.join(os.getcwd(), 'pipelines', PIPELINE_NAME)

os.makedirs(PIPELINE_ROOT, exist_ok=True)

context = InteractiveContext(pipeline_root=PIPELINE_ROOT)

Insight: 
* Mendefinisikan nama pipeline dan folder tempat artefak pipeline disimpan (`PIPELINE_ROOT`) dan membuat folder tersebut jika belum ada
* Menginisialisasi `InteractiveContext` agar pipeline bisa dijalankan langkah demi langkah di dalam notebook
* Metadata pipeline akan otomatis disimpan dalam SQLite di direktori pipeline

# Load Dataset

In [3]:
with open("data/insurance.csv", "r") as f:
    for _ in range(10):
        print(f.readline())

age,sex,bmi,children,smoker,region,charges

19,female,27.9,0,yes,southwest,16884.924

18,male,33.77,1,no,southeast,1725.5523

28,male,33,3,no,southeast,4449.462

33,male,22.705,0,no,northwest,21984.47061

32,male,28.88,0,no,northwest,3866.8552

31,female,25.74,0,no,southeast,3756.6216

46,female,33.44,1,no,southeast,8240.5896

37,female,27.74,3,no,northwest,7281.5056

37,male,29.83,2,no,northeast,6406.4107



Insight: 
* Memanggil dataset dan menampilkannya jika berhasil

# Load dataset ke pipeline

In [4]:
artifact = ExternalArtifact()
artifact.uri = '/app/data'

input_channel = channel_utils.as_channel([artifact])

example_gen = CsvExampleGen(input=input_channel)
context.run(example_gen)

2025-07-09 10:57:16.412160: W ml_metadata/metadata_store/rdbms_metadata_access_object.cc:484] No property is defined for the Type


ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 1
    outputs:
        examples: Channel(
            type_name: Examples
            artifacts: [Artifact(type_name: Examples, uri: /app/pipelines/insurance_pipeline/CsvExampleGen/examples/1, id: 2)]
        ))

Insight: 
* Pada tahap ini,  menggunakan komponen `CsvExampleGen` untuk membaca file `insurance.csv` dan mengubahnya menjadi format `TFRecord` yang digunakan di pipeline TFX.

# Generate Data Statistics (StatisticsGen)

In [5]:
statistics_gen = StatisticsGen(
    examples=example_gen.outputs['examples']
)
context.run(statistics_gen)

2025-07-09 10:57:25.731139: I tensorflow/core/platform/cpu_feature_guard.cc:143] Your CPU supports instructions that this TensorFlow binary was not compiled to use: SSE4.1 SSE4.2
2025-07-09 10:57:25.875240: I tensorflow/core/platform/profile_utils/cpu_utils.cc:102] CPU Frequency: 2592055000 Hz
2025-07-09 10:57:25.900352: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7fff6c22ec10 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2025-07-09 10:57:25.905560: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version


ExecutionResult(
    component_id: StatisticsGen
    execution_id: 2
    outputs:
        statistics: Channel(
            type_name: ExampleStatistics
            artifacts: [Artifact(type_name: ExampleStatistics, uri: /app/pipelines/insurance_pipeline/StatisticsGen/statistics/2, id: 3)]
        ))


Insight: 
* Di langkah ini, kita menggunakan `StatisticsGen` untuk menganalisis dataset dan menghasilkan statistik deskriptif (seperti distribusi nilai, missing value, dll).  
Statistik ini akan digunakan oleh `SchemaGen` untuk membuat skema otomatis dari data.

# Generate Schema (SchemaGen)

In [6]:
schema_gen = SchemaGen(
    statistics=statistics_gen.outputs['statistics']
)
context.run(schema_gen)

Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


2025-07-09 10:57:33.272001: W ml_metadata/metadata_store/rdbms_metadata_access_object.cc:484] No property is defined for the Type


ExecutionResult(
    component_id: SchemaGen
    execution_id: 3
    outputs:
        schema: Channel(
            type_name: Schema
            artifacts: [Artifact(type_name: Schema, uri: /app/pipelines/insurance_pipeline/SchemaGen/schema/3, id: 4)]
        ))

Insight:
* Komponen `SchemaGen` akan membaca statistik dari `StatisticsGen` dan menghasilkan schema otomatis.  
* Schema ini mencakup beberap informasi seperti kolom, tipe data, nilai unik, dan info missing value.
* Schema ini akan digunakan di tahap selanjutnya untuk validasi data dan transformasi fitur.

# Validasi Data (ExampleValidator)

In [7]:
example_validator = ExampleValidator(
    statistics=statistics_gen.outputs['statistics'],
    schema=schema_gen.outputs['schema']
)

context.run(example_validator)

ExecutionResult(
    component_id: ExampleValidator
    execution_id: 4
    outputs:
        anomalies: Channel(
            type_name: ExampleAnomalies
            artifacts: [Artifact(type_name: ExampleAnomalies, uri: /app/pipelines/insurance_pipeline/ExampleValidator/anomalies/4, id: 5)]
        ))

Insight:
* Komponen `ExampleValidator` digunakan untuk memvalidasi data terhadap schema yang sudah dihasilkan.  
* Jika ditemukan nilai yang tidak sesuai (missing, salah tipe, atau outlier), maka akan tercatat sebagai anomali.

# Transform (Feature Engineering)

In [8]:
transform = Transform(
    examples=example_gen.outputs['examples'],
    schema=schema_gen.outputs['schema'],
    module_file=os.path.abspath('transform_pipeline.py') 
)

context.run(transform)

Instructions for updating:
Schema is a deprecated, use schema_utils.schema_from_feature_spec to create a `Schema`


2025-07-09 10:57:35.615824: W ml_metadata/metadata_store/rdbms_metadata_access_object.cc:484] No property is defined for the Type


Instructions for updating:
Use ref() instead.


  name: "age"
  value_count {
    min: 1
    max: 1
  }
  type: INT
  presence {
    min_fraction: 1.0
    min_count: 1
  }
}
feature {
  name: "bmi"
  value_count {
    min: 1
    max: 1
  }
  type: FLOAT
  presence {
    min_fraction: 1.0
    min_count: 1
  }
}
feature {
  name: "charges"
  value_count {
    min: 1
    max: 1
  }
  type: FLOAT
  presence {
    min_fraction: 1.0
    min_count: 1
  }
}
feature {
  name: "children"
  value_count {
    min: 1
    max: 1
  }
  type: INT
  presence {
    min_fraction: 1.0
    min_count: 1
  }
}
feature {
  name: "region"
  value_count {
    min: 1
    max: 1
  }
  type: BYTES
  domain: "region"
  presence {
    min_fraction: 1.0
    min_count: 1
  }
}
feature {
  name: "sex"
  value_count {
    min: 1
    max: 1
  }
  type: BYTES
  domain: "sex"
  presence {
    min_fraction: 1.0
    min_count: 1
  }
}
feature {
  name: "smoker"
  value_count {
    min: 1
    max: 1
  }
  type: BYTES
  domain: "smoker"
  presence {
    min_fraction: 1.0
  

Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
Type is unsupported, or the types of the items don't match field type in CollectionDef. Note this is a warning and probably safe to ignore.
'Counter' object has no attribute 'name'
INFO:tensorflow:SavedModel written to: /app/pipelines/insurance_pipeline/Transform/transform_graph/5/.temp_path/tftransform_tmp/9a9f08c82380416abda12d393615316f/saved_model.pb
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
Type is unsupported, or the types of the items don't match field type in CollectionDef. Note this is a warning and probably safe to ignore.
'Counter' object has no attribute 'name'
INFO:tensorflow:SavedModel written to: /app/pipelines/insurance_pipeline/Transform/transform_graph/5/.temp_path/

  name: "age"
  value_count {
    min: 1
    max: 1
  }
  type: INT
  presence {
    min_fraction: 1.0
    min_count: 1
  }
}
feature {
  name: "bmi"
  value_count {
    min: 1
    max: 1
  }
  type: FLOAT
  presence {
    min_fraction: 1.0
    min_count: 1
  }
}
feature {
  name: "charges"
  value_count {
    min: 1
    max: 1
  }
  type: FLOAT
  presence {
    min_fraction: 1.0
    min_count: 1
  }
}
feature {
  name: "children"
  value_count {
    min: 1
    max: 1
  }
  type: INT
  presence {
    min_fraction: 1.0
    min_count: 1
  }
}
feature {
  name: "region"
  value_count {
    min: 1
    max: 1
  }
  type: BYTES
  domain: "region"
  presence {
    min_fraction: 1.0
    min_count: 1
  }
}
feature {
  name: "sex"
  value_count {
    min: 1
    max: 1
  }
  type: BYTES
  domain: "sex"
  presence {
    min_fraction: 1.0
    min_count: 1
  }
}
feature {
  name: "smoker"
  value_count {
    min: 1
    max: 1
  }
  type: BYTES
  domain: "smoker"
  presence {
    min_fraction: 1.0
  

  name: "age"
  type: INT
}
feature {
  name: "bmi"
  type: FLOAT
}
feature {
  name: "charges"
  type: FLOAT
}
feature {
  name: "children"
  type: INT
}
feature {
  name: "region"
  type: BYTES
}
feature {
  name: "sex"
  type: BYTES
}
feature {
  name: "smoker"
  type: BYTES
}
}), (<PCollection[Analyze/CreateSavedModel/BindTensors/ReplaceWithConstants.None] at 0x7fff8e7aff10>, BeamDatasetMetadata(dataset_metadata={'_schema': feature {
  name: "age"
  type: FLOAT
}
feature {
  name: "bmi"
  type: FLOAT
}
feature {
  name: "charges_xf"
  type: FLOAT
}
feature {
  name: "children"
  type: FLOAT
}
feature {
  name: "region"
  type: FLOAT
}
feature {
  name: "sex"
  type: FLOAT
}
feature {
  name: "smoker"
  type: FLOAT
}
}, deferred_metadata=<PCollection[Analyze/ComputeDeferredMetadata.None] at 0x7fff8e78a390>))) belongs to. Thus noop.


  name: "age"
  type: INT
}
feature {
  name: "bmi"
  type: FLOAT
}
feature {
  name: "charges"
  type: FLOAT
}
feature {
  name: "children"
  type: INT
}
feature {
  name: "region"
  type: BYTES
}
feature {
  name: "sex"
  type: BYTES
}
feature {
  name: "smoker"
  type: BYTES
}
}), (<PCollection[Analyze/CreateSavedModel/BindTensors/ReplaceWithConstants.None] at 0x7fff8e7aff10>, BeamDatasetMetadata(dataset_metadata={'_schema': feature {
  name: "age"
  type: FLOAT
}
feature {
  name: "bmi"
  type: FLOAT
}
feature {
  name: "charges_xf"
  type: FLOAT
}
feature {
  name: "children"
  type: FLOAT
}
feature {
  name: "region"
  type: FLOAT
}
feature {
  name: "sex"
  type: FLOAT
}
feature {
  name: "smoker"
  type: FLOAT
}
}, deferred_metadata=<PCollection[Analyze/ComputeDeferredMetadata.None] at 0x7fff8e78a390>))) belongs to. Thus noop.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:Assets written to: /app/pipelines/insurance_pipeline/Transform/transform_graph/5/.temp_path/tftransform_tmp/999d4dcd78eb4b1295a5cb0d164998dd/assets
INFO:tensorflow:SavedModel written to: /app/pipelines/insurance_pipeline/Transform/transform_graph/5/.temp_path/tftransform_tmp/999d4dcd78eb4b1295a5cb0d164998dd/saved_model.pb
value: "\n\013\n\tConst_3:0\022-vocab_compute_and_apply_vocabulary_vocabulary"

value: "\n\013\n\tConst_5:0\022/vocab_compute_and_apply_vocabulary_1_vocabulary"

value: "\n\013\n\tConst_7:0\022/vocab_compute_and_apply_vocabulary_2_vocabulary"

INFO:tensorflow:Saver not created because there are no variables in the graph to restore
value: "\n\013\n\tConst_3:0\022-vocab_compute_and_apply_vocabulary_vocabulary"

value: "\n\013\n\tConst_5:0\02

ExecutionResult(
    component_id: Transform
    execution_id: 5
    outputs:
        transform_graph: Channel(
            type_name: TransformGraph
            artifacts: [Artifact(type_name: TransformGraph, uri: /app/pipelines/insurance_pipeline/Transform/transform_graph/5, id: 6)]
        )
        transformed_examples: Channel(
            type_name: Examples
            artifacts: [Artifact(type_name: Examples, uri: /app/pipelines/insurance_pipeline/Transform/transformed_examples/5, id: 7)]
        ))

Insight: 
* Komponen Transform digunakan untuk melakukan feature engineering (scaling, encoding, dll) berdasarkan schema.
* Contoh: kolom age dan bmi di-scale, kolom sex dan smoker di-encode.


# Training Model (Trainer)

In [ ]:
trainer = Trainer(
    examples=transform.outputs['transformed_examples'],
    transform_graph=transform.outputs['transform_graph'],
    schema=schema_gen.outputs['schema'],
    train_args=trainer_pb2.TrainArgs(num_steps=100),
    eval_args=trainer_pb2.EvalArgs(num_steps=50),
    module_file='/app/trainer_pipeline.py',
    custom_executor_spec=executor_spec.ExecutorClassSpec(GenericExecutor)
)

context.run(trainer)

Epoch 1/5
 28/100 [=======>......................] - 1s 25ms/step - loss: 1.1792 - mae: 0.8329 - val_loss: 1.1362 - val_mae: 0.8052
INFO:tensorflow:Assets written to: /app/pipelines/insurance_pipeline/Trainer/model/15/serving_model_dir/assets


ExecutionResult(
    component_id: Trainer
    execution_id: 15
    outputs:
        model: Channel(
            type_name: Model
            artifacts: [Artifact(type_name: Model, uri: /app/pipelines/insurance_pipeline/Trainer/model/15, id: 17)]
        ))

Insight:
* 	Komponen Trainer digunakan untuk melakukan training model machine learning dengan dataset yang sudah di-transform.
* Model dibangun dengan Keras dan dilatih menggunakan fitur yang sudah diproses (hasil dari Transform), seperti:
	- age, bmi, children di-scale
	- sex, smoker, region di-encode
	- Target (label) charges_xf adalah hasil transformasi dari kolom charges dan digunakan sebagai label regresi.
* Model disimpan dalam format SavedModel TensorFlow ke direktori serving_model

# Evalusi Model

In [37]:
evaluator = Evaluator(
    examples=transform.outputs['transformed_examples'],
    model=trainer.outputs['model'],
    eval_config=config_pb2.EvalConfig(
        model_specs=[config_pb2.ModelSpec(label_key='charges_xf')],
        slicing_specs=[config_pb2.SlicingSpec()]
    )
)

context.run(evaluator)

2025-07-09 11:18:04.453287: W ml_metadata/metadata_store/rdbms_metadata_access_object.cc:484] No property is defined for the Type
2025-07-09 11:18:04.484892: W ml_metadata/metadata_store/rdbms_metadata_access_object.cc:484] No property is defined for the Type


ExecutionResult(
    component_id: Evaluator
    execution_id: 16
    outputs:
        evaluation: Channel(
            type_name: ModelEvaluation
            artifacts: [Artifact(type_name: ModelEvaluation, uri: /app/pipelines/insurance_pipeline/Evaluator/evaluation/16, id: 18)]
        )
        blessing: Channel(
            type_name: ModelBlessing
            artifacts: [Artifact(type_name: ModelBlessing, uri: /app/pipelines/insurance_pipeline/Evaluator/blessing/16, id: 19)]
        ))

Insight: 
* Komponen Evaluator digunakan untuk mengevaluasi performa model yang sudah dilatih.
* Evaluasi dilakukan menggunakan dataset hasil Transform (data yang sudah di-scale dan encode).

# Menjalankan Komponen Pusher

In [42]:
pusher = Pusher(
    model=trainer.outputs['model'],
    model_blessing=evaluator.outputs['blessing'],
    push_destination=tfx.proto.pusher_pb2.PushDestination(
        filesystem=tfx.proto.pusher_pb2.PushDestination.Filesystem(
            base_directory=os.path.join(os.getcwd(), 'serving_model')
        )
    )
)

context.run(pusher)

2025-07-09 11:28:54.667657: W ml_metadata/metadata_store/rdbms_metadata_access_object.cc:484] No property is defined for the Type


ExecutionResult(
    component_id: Pusher
    execution_id: 17
    outputs:
        pushed_model: Channel(
            type_name: PushedModel
            artifacts: [Artifact(type_name: PushedModel, uri: /app/pipelines/insurance_pipeline/Pusher/pushed_model/17, id: 20)]
        ))

Insight:
* Komponen Pusher digunakan untuk mendorong (deploy) model yang sudah "blessed" (lolos evaluasi) ke lokasi tertentu (misal: folder serving).
* Pusher hanya akan menyalin model ke folder tujuan jika model tersebut dinyatakan lulus oleh Evaluator (via model_blessing).
* Pada kasus ini, model disimpan ke folder `serving_model/`, yang bisa digunakan untuk proses serving ke production atau testing API.